\-----------------------

Correlated Subqueries

\-----------------------

Proposition: Write a query that returns each customers most recent order along with its associated order date and employee id.

  

This is a correlated subquery. Therefore the inner query also depends on attributes from the outer query.

In this case, the inner query is working on every row of the Sales.Order table from the outer query.  The inner query selects the maximum OrderId of all orders where the CustomerId matches the CustomerId of the row in the outer query that the inner query is working on.  If this maximum OrderId matches the OrderId of the outer query row, that row is included in the output table as that customers most recent order.

In [117]:
USE Northwinds2022TSQLV7;

SELECT CustomerId, OrderId, OrderDate, EmployeeId
FROM Sales.[Order] AS O1
WHERE orderid =
  (SELECT MAX(O2.OrderId)
   FROM Sales.[Order] AS O2
   WHERE O2.CustomerId = O1.CustomerId)
ORDER BY O1.CustomerId;

(89 rows affected)

Total execution time: 00:00:00.027

CustomerId,OrderId,OrderDate,EmployeeId
1,11011,2016-04-09,3
2,10926,2016-03-04,4
3,10856,2016-01-28,3
4,11016,2016-04-10,9
5,10924,2016-03-04,3
6,11058,2016-04-29,9
7,10826,2016-01-12,6
8,10970,2016-03-24,9
9,11076,2016-05-06,4
10,11048,2016-04-24,7


\-----------------------

Correlated Subqueries

\-----------------------

Proposition: Write a query that, for each order, returns the percentage of the total amount the customer has paid attributed to that specific order.

First, I created the view which the query uses. I had to do this since I could find the view in TSQLV4 but not in Northwinds2022TSQLV7.

This is a correlated subquery. Therefore the inner query also depends on attributes from the outer query. The inner query retains all of the rows from Sales.OrderValues which have the same CustomerId as the current outer query row we are working on and sums all of the cost values.  This result, which represents the customers total spending, is then used to divide the cost value of each individual row from the outer query.

In this case

In [118]:
USE Northwinds2022TSQLV7;
GO

CREATE VIEW Sales.OrderValues
WITH SCHEMABINDING
AS
SELECT O.OrderId, O.CustomerId, O.EmployeeId, O.ShipperId, O.OrderDate, O.RequiredDate, O.ShipToDate,  SUM(OD.Quantity) AS qty, CAST(SUM(OD.Quantity * OD.UnitPrice * (1 - OD.DiscountPercentage))
AS NUMERIC(12, 2)) AS val
FROM Sales.[Order] AS O
  JOIN Sales.[OrderDetail] AS OD
      ON O.OrderId = OD.OrderId
GROUP BY O.OrderId, O.CustomerId, O.EmployeeId, O.ShipperId, O.OrderDate, O.RequiredDate, O.ShipToDate;
GO

SELECT OrderId, CustomerId, val,
  CAST(100. * val / (SELECT SUM(O2.val)
                     FROM Sales.OrderValues AS O2
                     WHERE O2.CustomerId = O1.CustomerId)
       AS NUMERIC(5,2)) AS pct
FROM Sales.OrderValues AS O1
ORDER BY CustomerId, OrderId;

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure OrderValues, Line 2
There is already an object named 'OrderValues' in the database.

(830 rows affected)

Total execution time: 00:00:00.036

OrderId,CustomerId,val,pct
10643,1,814.50,19.06
10692,1,878.00,20.55
10702,1,330.00,7.72
10835,1,845.80,19.79
10952,1,471.20,11.03
11011,1,933.50,21.85
10308,2,88.80,6.33
10625,2,479.75,34.20
10759,2,320.00,22.81
10926,2,514.40,36.67


\--------

EXISTS  
\--------

Proposition: Write a query that returns a list of all the customers from Spain that have placed at least one order.

Our inner query returns all rows from the Sales.\[Order\] table where the CustomerId is the same as the outer query row's CustomerId.  In other words, the inner query will have results only if the customer has an order recorded in the Order table.  Then we use EXISTS to filter for only the customers that have placed an order.  If the customer did place an order, the inner query will have results, and EXISTS will return true.  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">If the customer did not place any orders, the inner query will return an empty result, and EXISTS will return false.&nbsp;&nbsp;</span>

In [104]:
USE Northwinds2022TSQLV7;

SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer] AS C
WHERE CustomerCountry = N'Spain'
  AND EXISTS
    (SELECT * FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId);

(4 rows affected)

Total execution time: 00:00:00.005

CustomerId,CustomerCompanyName
8,Customer QUHWH
29,Customer MDLWA
30,Customer KSLQF
69,Customer SIUIH


\--------

EXISTS  
\--------

Proposition: <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Write a query that returns a list of all the customers from Spain that have not placed any orders.</span>

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">This query works the same as the one above except it negates the EXISTS predicate with NOT to only return true if the inner query has an empty result, and return false if the inner query returns actual results.</span>

In [105]:
USE Northwinds2022TSQLV7;

SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer] AS C
WHERE CustomerCountry = N'Spain'
  AND NOT EXISTS
    (SELECT * FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId);

(1 row affected)

Total execution time: 00:00:00.004

CustomerId,CustomerCompanyName
22,Customer DTDMN


Chapter 4 Exercise 2

Proposition: Write a query that returns all of the orders that were placed by the customer who placed the most orders.

Here we use a subquery to find the customer who placed the most orders.  We use the results from the subquery to filter for only the orders placed by the corrrect customer/s.  Since multiple customers can be tied for most orders placed, we use TOP (1) WITH TIES in our inner query SELECT clause. This might return a multivalued tables so we must use IN instead of = in our outer query WHERE clause.

In [106]:
USE Northwinds2022TSQLV7;

SELECT CustomerId, OrderId, OrderDate, EmployeeId
FROM Sales.[Order]
WHERE CustomerId IN
    (SELECT TOP (1) WITH TIES O.CustomerId
    FROM Sales.[Order] AS O
    GROUP BY CustomerId
    ORDER BY COUNT(*) DESC);

(31 rows affected)

Total execution time: 00:00:00.006

CustomerId,OrderId,OrderDate,EmployeeId
71,10324,2014-10-08,9
71,10393,2014-12-25,1
71,10398,2014-12-30,2
71,10440,2015-02-10,4
71,10452,2015-02-20,8
71,10510,2015-04-18,6
71,10555,2015-06-02,6
71,10603,2015-07-18,8
71,10607,2015-07-22,5
71,10612,2015-07-28,1


Chapter 4 Exercise 4

Proposition: Write a query that returns all of the countries where customers live but no employees.

Here we use a subquery to find all the countries where employees live.  Then the outer query filters out all rows in the Sales.Customer table where the customers country is in the list of countries from the inner query.  We use the DISTINCT clause to ensure no duplicates in our resulting table.

In [107]:
USE Northwinds2022TSQLV7;

SELECT DISTINCT CustomerCountry
FROM Sales.[Customer]
WHERE CustomerCountry NOT IN
    (SELECT EmployeeCountry 
    FROM HumanResources.[Employee])

(19 rows affected)

Total execution time: 00:00:00.005

CustomerCountry
Argentina
Austria
Belgium
Brazil
Canada
Denmark
Finland
France
Germany
Ireland


\---------------

Derived Tables

\---------------

Proposition: Write a query that returns a list of all the customers from the USA with their company name.

A derived table is used just to show the syntax.

In [108]:
USE Northwinds2022TSQLV7;

SELECT *
FROM (SELECT CustomerId, CustomerCompanyName
    FROM Sales.[Customer]
    WHERE CustomerCountry = N'USA') AS USACust

(13 rows affected)

Total execution time: 00:00:00.004

CustomerId,CustomerCompanyName
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


\--------------------------

Assigning Column Aliases

\--------------------------

Proposition: Write a query that, for every year, returns the total number of customers that placed at least one order during that year.

The commented out version doesn't work.  This is because the GROUP BY clause refers to an alias which is only defined in the SELECT clause.  However, the SELECT clause is executed after the GROUP BY clause.

One solution is to just use YEAR(orderdate) in the GROUP BY clause instead of the alias.  This is shown in the middle valid query.

Another solution is to assign the column alias to a derived table.  Now, the alias is available to be access anywhere in the outer query.  This is shown in the first valid query.

A different way to assign the alias in the derived table is shown in the last valid query.

In [109]:
USE Northwinds2022TSQLV7;

/*
SELECT
  YEAR(OrderDate) AS orderyear,
  COUNT(DISTINCT CustomerId) AS numcusts
FROM Sales.[Order]
GROUP BY orderyear;
*/

SELECT orderyear, COUNT(DISTINCT CustomerId) AS numcusts
FROM (SELECT YEAR(OrderDate) AS orderyear, CustomerId
      FROM Sales.[Order]) AS D
GROUP BY orderyear;

SELECT YEAR(orderdate) AS orderyear, COUNT(DISTINCT CustomerId) AS numcusts
FROM Sales.[Order]
GROUP BY YEAR(orderdate);

SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM (SELECT YEAR(orderdate), CustomerId
      FROM Sales.[Order]) AS D(orderyear, custid)
GROUP BY orderyear;

(3 rows affected)

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.013

orderyear,numcusts
2014,67
2015,86
2016,81


orderyear,numcusts
2014,67
2015,86
2016,81


orderyear,numcusts
2014,67
2015,86
2016,81


\--------------

ENCRYPTION

\--------------

We create a view which stores all the customers from the USA and their information.  We use the OBJECT\_DEFINITION function to get the definition of the view.  But then, if we use the ENCRYPTION attribute, the definition of the view is no longer available to any user.  Instead the OBJECT\_DEFINITION function returns NULL.

In the final line, we use sp\_helptext to get the view definition.  Since the view is encrypted, we get a message saying so instead of the actual definition.

In [110]:
USE Northwinds2022TSQLV7;
GO

ALTER VIEW Sales.[USACusts]
AS
SELECT
  CustomerId, CustomerCompanyName, CustomerContactName, CustomerContactTitle, CustomerAddress,
  CustomerCity, CustomerRegion, CustomerPostalCode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber
FROM Sales.[Customer]
WHERE CustomerCountry = N'USA';
GO

SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));
GO

ALTER VIEW Sales.[USACusts] WITH ENCRYPTION
AS
SELECT
  CustomerId, CustomerCompanyName, CustomerContactName, CustomerContactTitle, CustomerAddress,
  CustomerCity, CustomerRegion, CustomerPostalCode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber
FROM Sales.[Customer]
WHERE CustomerCountry = N'USA';
GO

SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));
GO

EXEC sp_helptext 'Sales.USACusts';
GO

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Commands completed successfully.

(1 row affected)

The text for object 'Sales.USACusts' is encrypted.

Total execution time: 00:00:00.026

(No column name)
"CREATE VIEW Sales.[USACusts] AS SELECT CustomerId, CustomerCompanyName, CustomerContactName, CustomerContactTitle, CustomerAddress, CustomerCity, CustomerRegion, CustomerPostalCode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber FROM Sales.[Customer] WHERE CustomerCountry = N'USA';"


(No column name)
NULL


Chapter 5 Exercise 4

Proposition: Write a query that shows Patricia Doyle's supervisory chain.

Here we use a recursive CTE.  EmpCTE has a anchor\_member that is invoked once starting with the employee with id = 9 (Patricia Doyle).  It also has a recursive\_member which invokes EmpCTE itself, making it recursive.  The recursive query matches the previous call's ManagerId to the EmployeeId.

In [111]:
USE Northwinds2022TSQLV7;

WITH EmpCTE AS
(
    SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName
    FROM HumanResources.[Employee]
    WHERE EmployeeId = 9

    UNION ALL

    SELECT P.EmployeeId, P.EmployeeManagerId, P.EmployeeFirstName, P.EmployeeLastName
    FROM EmpCTE AS C
        INNER JOIN HumanResources.[Employee] AS P
            ON C.EmployeeManagerId = P.EmployeeId
)
SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName
FROM EmpCTE

(4 rows affected)

Total execution time: 00:00:00.004

EmployeeId,EmployeeManagerId,EmployeeFirstName,EmployeeLastName
9,5,Patricia,Doyle
5,2,Sven,Mortensen
2,1,Don,Funk
1,NULL,Sara,Davis


Chapter 5 Exercise 6

Proposition: Given a supplier and integer value n, write a query that returns the n most expensive products provided by the supplier.

Here we use an inline TVF that takes in two parameters: an integer representing the SupplierId and an integer representing the number of most expensive products we want to include.  The TVF returns a table which filtered from the Production.\[Product\] table for only the rows which have the correct SupplierId and then ordered them by UnitPrice, taking the top n results.  Then we query the TVF with parameters 5 and 2.

In [112]:
USE Northwinds2022TSQLV7;
GO

CREATE FUNCTION Production.TopProducts
    (@supid AS INT, @n AS INT)
    RETURNS TABLE
AS
RETURN
    SELECT TOP (@n) ProductId, ProductName, UnitPrice
    FROM Production.[Product]
    WHERE SupplierId = @supid
    ORDER BY UnitPrice DESC;
GO

SELECT * FROM Production.TopProducts(5, 2)
GO

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure TopProducts, Line 2
There is already an object named 'TopProducts' in the database.

(2 rows affected)

Total execution time: 00:00:00.017

ProductId,ProductName,UnitPrice
12,Product OSFNS,38.00
11,Product QMVUN,21.00
